In [2]:
import pandas as pd
import nltk as nltk

import ssl
# Desactiva la verificación SSL > tengo que hacer esto para poder realizar las descargas necesarias
ssl._create_default_https_context = ssl._create_unverified_context
nltk.download('vader_lexicon')
nltk.download('punkt')
nltk.download('stopwords')

#SentimentIntensityAnalyzer es una clase proporcionada por la librería NLTK para realizar análisis de sentimientos.
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk import tokenize

df_reviews= pd.read_parquet("Archivos_MVP/reviews_mvp.parquet")
df_reviews.head(6)

#Me quedo con la data de reviews, y convierte todo strings
reviews=df_reviews["review"].astype(str)

#Creo una lista con las reviews
lista_reviews= list(reviews)   #Genero lista a partir de la columna reviews
lista_reviews[10] #compruebo lista

#Realizo el analisis de sentimientos  con SentimentIntensityAnalyzer()

clasificacion = []    #Lista donde se van a almacenar los resultados del analisis de sentimientos 

#Se instancia al objeto de SentimentIntensityAnalyzer() de la librería NLTK, que se utilizará para realizar el análisis de sentimientos
analisis_sentim = SentimentIntensityAnalyzer()   

# Clasificación para cada frase en la lista de reviews "Positivo/Neutral/Negativo"
for review in lista_reviews:
    
    # polarity_scores() devuelve un diccionario que contiene varios puntajes relacionados con la polaridad de la review
    score = analisis_sentim.polarity_scores(review) 
    #print(score)
    #La columna de compound me va a asignar un resumen general de la puntuación
    if score['compound'] > 0:
        clasificacion.append('Positivo')
    elif score['compound'] == 0:
        clasificacion.append('Neutral')
    else:
        clasificacion.append('Negativo')

#Clasificacion ahora es una lista que tiene todas las clasificaciones de las reviews.
clasificacion[1000]


#Convierte la lista en un df para agregarlo luego y asociar a cada review con su puntaje
sentimientos= pd.Series(clasificacion)
sentimientos.value_counts()
# Asigna la lista de clasificación a la nueva columna en el DataFrame 'df_reviews': Col sent_analisis
df_reviews["sent_analisis"]=sentimientos

df_reviews.sent_analisis.value_counts()
df_reviews.review.value_counts()
df_reviews.info()
df_reviews.head(5)


#Elimino la columna que tiene las reviews dado que no voy a necesitarlas.
df_reviews.drop(columns=['review'], inplace=True)
df_reviews.head(5)

#Me quedo con un 25% de los datos de forma aleatoria para el MVP y lo guardo en formato parquet para que ocupe menos lugar

sentimientos_mvp=df_reviews.sample(frac=0.25, random_state=42)

sentimientos_mvp.to_parquet("Archivos_MVP/sentimientos_mvp.parquet", index=False)

#Paso el archivo a un parquet
#df_reviews.to_parquet("Archivos_/reviews_con_sentimientos",index=False)

sentimientos_mvp.head(3)


[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/teresalaura/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/teresalaura/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/teresalaura/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14615 entries, 0 to 14614
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   user_id        14615 non-null  object 
 1   posted         12148 non-null  float64
 2   item_id        14608 non-null  object 
 3   review         14608 non-null  object 
 4   sent_analisis  14615 non-null  object 
dtypes: float64(1), object(4)
memory usage: 571.0+ KB


,user_id,posted,item_id,sent_analisis
11184,76561198009177886,2014.0,10500,Negativo
6466,76561198066371009,2014.0,221640,Positivo
10304,rexsniper,2013.0,219740,Neutral
